In [1]:
# Import packages and set OPENAI Key access
import nltk
import json
#import langchain as lc
from getpass import getpass
import os
from langchain.chat_models import ChatOpenAI
#from langchain.chains import create_extraction_chain
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
OPENAI_ORGANIZATION = os.environ['OPENAI_ORGANIZATION']
chatGPT4 = ChatOpenAI(openai_api_key=OPENAI_API_KEY, openai_organization= OPENAI_ORGANIZATION, temperature=0, model="gpt-4")
chatGPT35 = ChatOpenAI(openai_api_key=OPENAI_API_KEY, openai_organization= OPENAI_ORGANIZATION, temperature=0, model="gpt-3.5-turbo")


In [2]:
# Load 1st set of Promed data
D1 = json.load(open('../../WHO_DON/Data/ProMED/promed_1.json'))

In [3]:
# Headers that demonstrate that the extrations can't be made by parsing as the disease outbreak location is not 
# always at the same position. Need to use LLMs
import numpy as np
print(D1[0]['header'][0])
print(D1[22]['header'][0])
print(D1[100]['header'][0])
print(D1[25]['header'][0])


Published Date: 2007-03-21 18:00:01 EDT
Subject: PRO/AH/EDR> Avian influenza (57): Kuwait, Myanmar
Archive Number: 20070321.1000
Published Date: 2007-11-30 08:00:21 EST
Subject: PRO/AH/EDR> Undiagnosed deaths - Zambia (02): pneumonic plague susp.
Archive Number: 20071130.3862
Published Date: 2005-02-10 18:50:00 EST
Subject: PRO/PL> Soybean rust, dry beans - South Africa
Archive Number: 20050210.0452
Published Date: 2004-04-08 19:50:00 EDT
Subject: PRO/EDR> Gastroenteritis, fatal - Zambia: RFI
Archive Number: 20040408.0954


In [41]:
# ##Approach 1
# from langchain.chat_models import ChatOpenAI
# from langchain.chains import create_extraction_chain
# schema = {
#     "properties": {
#         "disease outbreak": {"type": "string"},
#     },
#     "required": ["disease outbreak"],
# }

# # Input 
# inp = "Published Date: 2018-10-17 08:11:34 EDT Subject: PRO/AH/EDR> Undiagnosed die-off, hare - UK (02): (England) myxomatosis susp Archive Number: 20181017.6096127"
# # Run chain
# # Use model="gpt-3.5-turbo" for gpt 3.5. Its cheaper.
# llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0, model="gpt-4")
# chain = create_extraction_chain(schema, llm)
# chain.run(inp)
# #Didn't work well

[{'disease outbreak': 'Undiagnosed die-off, hare - UK (02): (England) myxomatosis susp'}]

In [ ]:
# ## Example 2
# inp = "Published Date: 2018-09-30 07:04:05 EDT Subject: PRO/EDR> Meningitis, meningococcal - USA (02): (CA) college, sg B Archive Number: 20180930.6061362"
# llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, temperature=0, model="gpt-4")
# chain = create_extraction_chain(schema, llm)
# chain.run(inp)

In [4]:
# Approach 2
inp = "Published Date: 2018-10-17 08:11:34 EDT Subject: PRO/AH/EDR> Undiagnosed die-off, hare - UK (02): (England) myxomatosis susp Archive Number: 20181017.6096127"
#chat([HumanMessage(content="Extract the disease outbreak from the following text. Only output the outbreak name. If the outbreak is known by other names give a comma separated list:" + D1[25]['header'][0])])
chatGPT35([HumanMessage(content="Extract the disease outbreak from the following text. " + "Only output the outbreak name." + inp )])
## This approach gives better results

AIMessage(content='myxomatosis', additional_kwargs={}, example=False)

In [46]:
# Example 2
inp = "Published Date: 2018-09-30 07:04:05 EDT Subject: PRO/EDR> Meningitis, meningococcal - USA (02): (CA) college, sg B Archive Number: 20180930.6061362"
chatGPT35([HumanMessage(content="Extract the disease outbreak from the following text. " + "Only output the outbreak name." + inp )])

AIMessage(content='Meningitis, meningococcal', additional_kwargs={}, example=False)

In [37]:
# To see how good are LLMs for picking alternate names. Maybe not the best example.
chat([HumanMessage(content="Give alternative names of the following disease outbreak with confidence:" + "Sars-CoV-2")])

AIMessage(content='1. COVID-19\n2. Coronavirus\n3. Novel coronavirus\n4. Wuhan coronavirus\n5. SARS-CoV-2 pandemic\n6. The 2019 coronavirus outbreak\n7. The global coronavirus crisis\n8. The Great Lockdown\n9. The COVID-19 pandemic', additional_kwargs={}, example=False)

In [5]:
xdata = np.array([(1, 2, 3), (4, 5, 6), (7, 8, 9)],
                dtype=[("a", "i4"), ("b", "i4"), ("c", "i4")])

In [19]:
import pandas as pd
import datetime
df = pd.DataFrame({'floa': [1.0],
                   'in': [1],
                   'datetim': [pd.Timestamp('20180310')],
                   'strin': ['foo']}, dtype=["float64", "int64", "datetime64[ns]", "string[pyarrow]"])
df
dti = pd.to_datetime(
    ["1/1/2018", np.datetime64("2018-01-01"), datetime.datetime(2018, 1, 1)]
)

TypeError: Field elements must be 2- or 3-tuples, got ''float64''

In [17]:
df.dtypes

floa              float64
in                  int64
datetim    datetime64[ns]
strin              object
dtype: object